# 패키지 로딩

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score, cross_validate
from sklearn.tree import DecisionTreeClassifier # 의사결정나무모델
from sklearn.metrics import accuracy_score  # 정확도 측정

# None Cross Validation

In [2]:
iris = load_iris() # 분류분석

x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=10)

model = DecisionTreeClassifier(random_state=1) # 의사결정나무모델
model.fit(x_train, y_train)
y_hat = model.predict(x_test)
print(f'정확도: {accuracy_score(y_test, y_hat):.2f}')  # (정답, 예측값)

# 교차검증했을 때보다 정확도가 높을 시 과적합 발생한 경우

정확도: 0.98


# K-Fold Cross Validation

In [3]:
print(iris.data.shape)
print(iris.target)

(150, 4)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [9]:
k_fold = KFold(n_splits=5)#, shuffle=True) # n_splits: 분할 개수
# 학습에는 k-1개 데이터셋 사용, 검증에는 1개 데이터셋 사용
# shuffle: 차례대로 분할해서 가져오지 않고 임의로 가져옴(종속변수가 편중되는 문제때문에 사용)

x = iris.data
y = iris.target
k = 0  # 교차검증의 횟수를 저장할 변수
cv_acc = []  # 각 교차검증마다 정확도를 저장할 변수

model = DecisionTreeClassifier(random_state=1)
for train_idx, test_idx in k_fold.split(x):  # kfold.split: 데이터를 5등분, 각 학습, 검증에 쓰이는 데이터의 인덱스 값을 반환
    k += 1
    print(f'{k}번째 검증 데이터 셋')
    print('Train:', train_idx)  # 학습데이터의 인덱스
    print('Test:', test_idx)   # 검증데이터의 인덱스
    x_train, x_test = x[train_idx], x[test_idx]  # 학습/검증데이터에 가져온 인덱스 값으로 할당
    y_train, y_test = y[train_idx], y[test_idx]

    model.fit(x_train, y_train)
    y_hat = model.predict(x_test)
    acc = accuracy_score(y_test, y_hat)
    print(f'{k}번째 교차검증의 정확도: {acc:.2f}')
    cv_acc.append(acc)
    print('='*70)

print()
print(f'평균 정확도: {np.mean(cv_acc):.2f}')

1번째 검증 데이터 셋
Train: [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
Test: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
1번째 교차검증의 정확도: 1.00
2번째 검증 데이터 셋
Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 

# Stratified KFold Cross Validation

In [11]:
stf_kfold = StratifiedKFold(n_splits=5)
# StratifiedKFold: 종속변수를 분할한 데이터마다 고르게 분포되도록
x = iris.data
y = iris.target
k = 0 
cv_acc = [] 

model = DecisionTreeClassifier(random_state=1)
for train_idx, test_idx in stf_kfold.split(x, y): # split(독립변수, 종속변수)  
    k += 1
    print(f'{k}번째 검증 데이터 셋')
    print('Train:', train_idx)  
    print('Test:', test_idx)   
    x_train, x_test = x[train_idx], x[test_idx]  
    y_train, y_test = y[train_idx], y[test_idx]

    model.fit(x_train, y_train)
    y_hat = model.predict(x_test)
    acc = accuracy_score(y_test, y_hat)
    print(f'{k}번째 교차검증의 정확도: {acc:.2f}')
    cv_acc.append(acc)
    print('='*70)

print()
print(f'평균 정확도: {np.mean(cv_acc):.2f}')

1번째 검증 데이터 셋
Train: [ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
Test: [  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109]
1번째 교차검증의 정확도: 0.97
2번째 검증 데이터 셋
Train: [  0   1   2   3   4   5   6   7   8   9  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  9

# cross_val_score()
- estimator: 학습할 모델 객체
- x: 독립변수 데이터 셋
- y: 종속변수 데이터 셋
- cv: 교차검증 k값 (default:5), 또는 교차검증 객체

In [12]:
score = cross_val_score(model, x, y)
# 모델이 분류모형이기 때문에 알아서 계층화된 kfold교차검증 적용 
print('score:', score)
print(f'평균 정확도: {np.mean(score):.2f}')

score: [0.96666667 0.96666667 0.9        1.         1.        ]
평균 정확도: 0.97


# cross_validate()
- 매 검증마다 소요된 학습 시간과 검증 시간을 딕셔너리로 반환
- 여러가지 평가지표를 리스트 형태로 전달 가능

In [14]:
import sklearn
sklearn.metrics.SCORERS.keys() # 회귀, 분류모형에 사용가능한 평가지표 리스트

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weig

In [15]:
result = cross_validate(model, x, y, cv=5, scoring=['accuracy', 'precision_macro', 'roc_auc_ovr'])  # scoring: 활용하고자 하는 평가지표
for key, val in result.items():
    print('평가지표:', key)
    print(f'평균값: {np.mean(val):.2f}')
    print('-'*30)

# fit/score time: 학습, 평가하는데 걸린 시간

평가지표: fit_time
평균값: 0.00
------------------------------
평가지표: score_time
평균값: 0.01
------------------------------
평가지표: test_accuracy
평균값: 0.97
------------------------------
평가지표: test_precision_macro
평균값: 0.97
------------------------------
평가지표: test_roc_auc_ovr
평균값: 0.97
------------------------------
